## This notbeook is for using the local ollama to generate the summarization of a webapge

In [ ]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# this class extracts/parses the webpage

class Website:

    def __init__(self, url):
        self.url = url

        #Creating a headless browser
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--enable-javascript")
        chrome_options.add_argument("--enable-cookies")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)

        if "Just a moment..." in driver.title:
            print("Cloudflare detected. Waiting...")
            time.sleep(5)

        element = WebDriverWait(driver,10)
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        
        page_source = driver.page_source
        driver.quit()
        
        soup = BeautifulSoup(page_source, 'html.parser')

        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
# lets call the website class to get the extracted content

extracted_text = Website("https://en.wikipedia.org/wiki/Pushpa_2:_The_Rule")
print(extracted_text.title)
print(extracted_text.text)

In [ ]:
# Now that we have the extracted content of the webage. Lets create necessary variables to call the ollama

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
# lets generate the prompts to give the context to the LLM

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown. Only use the important details from the context"

user_prompt = f"You are looking at a website titled {extracted_text.title}"
user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
user_prompt += extracted_text.text
messages = [
    {"role":"system", "content": system_prompt},
    {"role":"user", "content":user_prompt}
]

In [ ]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

## this is the end of the webapge summarization.